In [8]:
# CELL 1 — setup
import os
os.makedirs("data", exist_ok=True)
os.makedirs("modules", exist_ok=True)
os.makedirs("outputs", exist_ok=True)

import pandas as pd
print("Folders created and pandas version:", pd.__version__)

Folders created and pandas version: 2.2.2


In [9]:
# CELL 2 — sample dataset
data = {
    "product": ["rice", "sugar", "milk", "maggie", "bread", "oil", "salt", "tea", "coffee", "biscuits"],
    "price": [45, 50, 35, 14, 30, 120, 20, 200, 350, 30],
    "unit": ["kg","kg","litre","packet","packet","litre","kg","kg","kg","packet"],
    "sku": ["RICE_001","SUG_001","MLK_001","MAGGIE_001","BRD_001","OIL_001","SLT_001","TEA_001","COF_001","BIS_001"]
}
df = pd.DataFrame(data)
df.to_csv("data/products.csv", index=False)
df

,product,price,unit,sku
0,rice,45,kg,RICE_001
1,sugar,50,kg,SUG_001
2,milk,35,litre,MLK_001
3,maggie,14,packet,MAGGIE_001
4,bread,30,packet,BRD_001
5,oil,120,litre,OIL_001
6,salt,20,kg,SLT_001
7,tea,200,kg,TEA_001
8,coffee,350,kg,COF_001
9,biscuits,30,packet,BIS_001


In [10]:
# CELL 3 — search function (supports partial match)
import pandas as pd

def load_products():
    return pd.read_csv("data/products.csv")

def search_product(query):
    df = load_products()
    q = str(query).strip().lower()
    # exact match
    exact = df[df['product'].str.lower() == q]
    if not exact.empty:
        return exact.to_dict(orient='records')
    # partial contains (word inside product)
    contains = df[df['product'].str.lower().str.contains(q)]
    if not contains.empty:
        return contains.to_dict(orient='records')
    # token matching: check each word in query
    tokens = q.split()
    matched = df[df['product'].str.lower().apply(lambda p: any(t in p for t in tokens))]
    if not matched.empty:
        return matched.to_dict(orient='records')
    return []

# quick local test
print("search 'rice' ->", search_product("rice"))
print("search 'mil' ->", search_product("mil"))  # partial

search 'rice' -> [{'product': 'rice', 'price': 45, 'unit': 'kg', 'sku': 'RICE_001'}]
search 'mil' -> [{'product': 'milk', 'price': 35, 'unit': 'litre', 'sku': 'MLK_001'}]


In [11]:
# CELL 4 — cart and billing calculation
from datetime import datetime
from tabulate import tabulate

def add_to_cart(cart, sku, qty=1):
    df = load_products()
    row = df[df['sku']==sku]
    if row.empty:
        return False, "SKU not found"
    item = row.iloc[0].to_dict()
    item['qty'] = int(qty)
    item['line_total'] = item['qty'] * item['price']
    cart.append(item)
    return True, item

def compute_total(cart):
    total = sum(x['line_total'] for x in cart)
    return total

def cart_to_table(cart):
    if not cart:
        return "Cart empty"
    table = [(i+1, x['product'], x['sku'], x['qty'], x['unit'], x['price'], x['line_total']) for i,x in enumerate(cart)]
    return tabulate(table, headers=["#", "Product", "SKU", "Qty", "Unit", "Price", "LineTotal"], tablefmt="github")

# quick demo
cart = []
add_to_cart(cart, "RICE_001", 2)
add_to_cart(cart, "MLK_001", 3)
print(cart_to_table(cart))
print("Total =", compute_total(cart))

|   # | Product   | SKU      |   Qty | Unit   |   Price |   LineTotal |
|-----|-----------|----------|-------|--------|---------|-------------|
|   1 | rice      | RICE_001 |     2 | kg     |      45 |          90 |
|   2 | milk      | MLK_001  |     3 | litre  |      35 |         105 |
Total = 195


In [12]:
# CELL 5 — simple rule-based assistant
def assistant(user_input, cart=None):
    if cart is None:
        cart = []
    txt = str(user_input).lower()
    # price request
    if any(kw in txt for kw in ["price of", "price", "cost of", "cost"]):
        # extract product name heuristically
        for kw in ["price of", "cost of"]:
            if kw in txt:
                prod = txt.split(kw)[-1].strip()
                break
        else:
            prod = txt.replace("price","").replace("cost","").strip()
        res = search_product(prod)
        return {"type":"price","query":prod,"result":res}
    # show product / find
    if any(kw in txt for kw in ["show", "find", "search", "do you have", "stock"]):
        # remove keywords
        for k in ["show", "find", "search", "do you have", "stock", "is there", "list"]:
            txt = txt.replace(k,"")
        q = txt.strip()
        res = search_product(q)
        return {"type":"search","query":q,"result":res}
    # add to cart e.g. "add 2 rice" or "add rice 2"
    if txt.startswith("add ") or " add " in txt:
        # crude parse: look for a number
        parts = txt.split()
        qty = 1
        sku = None
        # find numeric token
        nums = [p for p in parts if p.isdigit()]
        if nums:
            qty = int(nums[0])
        # find product word (non-keyword)
        words = [p for p in parts if p not in ("add","to","cart","please")]
        # assume last non-numeric word is product
        prod_words = [w for w in words if not w.isdigit()]
        prod = " ".join(prod_words[1:]) if len(prod_words)>1 else (prod_words[-1] if prod_words else "")
        found = search_product(prod)
        if found:
            # pick first match SKU
            sku = found[0]['sku']
            ok, item = add_to_cart(cart, sku, qty)
            return {"type":"add","product":prod,"qty":qty,"result":item}
        else:
            return {"type":"add","product":prod,"qty":qty,"result":[]}
    # show cart
    if "show cart" in txt or "view cart" in txt or txt.strip()=="cart":
        return {"type":"cart","cart":cart,"table":cart_to_table(cart),"total":compute_total(cart)}
    return {"type":"unknown","text":txt, "msg":"I can help with product search, price lookup, add to cart, and show cart."}

# quick assistant demo
cart_demo = []
print(assistant("price of rice", cart_demo))
print(assistant("add 2 rice", cart_demo))
print(assistant("cart", cart_demo))

{'type': 'price', 'query': 'rice', 'result': [{'product': 'rice', 'price': 45, 'unit': 'kg', 'sku': 'RICE_001'}]}
{'type': 'add', 'product': 'rice', 'qty': 2, 'result': {'product': 'rice', 'price': 45, 'unit': 'kg', 'sku': 'RICE_001', 'qty': 2, 'line_total': 90}}
{'type': 'cart', 'cart': [{'product': 'rice', 'price': 45, 'unit': 'kg', 'sku': 'RICE_001', 'qty': 2, 'line_total': 90}], 'table': '|   # | Product   | SKU      |   Qty | Unit   |   Price |   LineTotal |\n|-----|-----------|----------|-------|--------|---------|-------------|\n|   1 | rice      | RICE_001 |     2 | kg     |      45 |          90 |', 'total': 90}


In [19]:
# PHASE 2 — Billing PDF + Sales logging
import pandas as pd
from reportlab.lib.pagesizes import A4
from reportlab.pdfgen import canvas
from datetime import datetime
import uuid
import os

os.makedirs("outputs", exist_ok=True)

def generate_bill_pdf(cart, shop_name="Santosh Kirana Store", invoice_id=None):
    if invoice_id is None:
        invoice_id = str(uuid.uuid4())[:8]
    now = datetime.now().strftime("%Y-%m-%d_%H%M%S")
    filename = f"outputs/bill_{invoice_id}_{now}.pdf"
    c = canvas.Canvas(filename, pagesize=A4)
    width, height = A4
    c.setFont("Helvetica-Bold", 16)
    c.drawString(50, height-50, shop_name)
    c.setFont("Helvetica", 10)
    c.drawString(50, height-70, f"Invoice ID: {invoice_id}")
    c.drawString(50, height-85, f"Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    # header
    y = height-120
    c.drawString(50, y, "No")
    c.drawString(100, y, "Product")
    c.drawString(300, y, "Qty")
    c.drawString(360, y, "Unit Price")
    c.drawString(450, y, "Line Total")
    y -= 20
    total = 0
    for i,item in enumerate(cart):
        c.drawString(50, y, str(i+1))
        c.drawString(100, y, str(item['product']))
        c.drawString(300, y, str(item['qty']))
        c.drawString(360, y, str(item['price']))
        c.drawString(450, y, str(item['line_total']))
        total += item['line_total']
        y -= 18
        if y < 60:
            c.showPage()
            y = height-50
    # total block
    c.setFont("Helvetica-Bold", 12)
    c.drawString(360, y-10, "Total:")
    c.drawString(450, y-10, str(total))
    c.showPage()
    c.save()
    return filename, invoice_id, total

def log_sale(invoice_id, total, filename):
    os.makedirs("outputs", exist_ok=True)
    log_path = "outputs/sales_log.csv"
    df = pd.DataFrame([{'invoice_id': invoice_id, 'total': total, 'file': filename, 'timestamp': datetime.now()}])
    if os.path.exists(log_path):
        df.to_csv(log_path, mode='a', header=False, index=False)
    else:
        df.to_csv(log_path, index=False)

# Quick test (run these after defining functions)
cart = [{'product':'rice','qty':2,'price':45,'line_total':90}, {'product':'milk','qty':1,'price':25,'line_total':25}]
filename, invoice_id, total = generate_bill_pdf(cart, shop_name="Demo Grocery")
log_sale(invoice_id, total, filename)
print("Generated:", filename, "Total:", total)

Generated: outputs/bill_67d7164f_2025-11-15_090347.pdf Total: 115


In [20]:
!ls -l outputs

total 12
-rw-r--r-- 1 root root 1805 Nov 15 09:03 bill_67d7164f_2025-11-15_090347.pdf
-rw-r--r-- 1 root root 1806 Nov 15 09:00 bill_bc44276a_2025-11-15_090055.pdf
-rw-r--r-- 1 root root  200 Nov 15 09:03 sales_log.csv


In [24]:
# PHASE 3 — integrate assistant + billing
def finalize_purchase(cart, shop_name="Santosh Kirana Store"):
    """
    Takes current cart (list of items with qty, price, line_total) and:
      - generates a PDF invoice
      - logs the sale
      - returns filename, invoice_id, total
    """
    if not cart:
        return {"ok": False, "msg": "Cart is empty"}
    filename, invoice_id, total = generate_bill_pdf(cart, shop_name=shop_name)
    log_sale(invoice_id, total, filename)
    # Optionally clear cart in caller
    return {"ok": True, "file": filename, "invoice_id": invoice_id, "total": total}

# Example usage with your existing assistant/cart:
result = finalize_purchase(cart)
print(result)

{'ok': True, 'file': 'outputs/bill_c90767b2_2025-11-15_090738.pdf', 'invoice_id': 'c90767b2', 'total': 115}


In [25]:
!git config --global user.email "nka150701@gmail.com"
!git config --global user.name "Nikhil Kumar Agrawal"
!git init
!git add AI_Business_Automation_Agent.ipynb data/products.csv README.txt .gitignore
!git commit -m "Initial commit: Phase1 notebook, data, README"
!git remote add origin https://YOUR_TOKEN@github.com/YOUR_GITHUB_USERNAME/ai-business-automation-agent.git
!git branch -M main
!git push -u origin main
!git remote remove origin

hint: Using 'master' as the name for the initial branch. This default branch name
hint: is subject to change. To configure the initial branch name to use in all
hint: of your new repositories, which will suppress this warning, call:
hint: 
hint: 	git config --global init.defaultBranch <name>
hint: 
hint: Names commonly chosen instead of 'master' are 'main', 'trunk' and
hint: 'development'. The just-created branch can be renamed via this command:
hint: 
hint: 	git branch -m <name>
Initialized empty Git repository in /content/.git/
fatal: pathspec 'AI_Business_Automation_Agent.ipynb' did not match any files
On branch master

Initial commit

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	.config/
	data/
	outputs/
	sample_data/

nothing added to commit but untracked files present (use "git add" to track)
error: src refspec main does not match any
error: failed to push some refs to 'https://github.com/YOUR_GITHUB_USERNAME/ai-business-automation-agent.git'